https://scikit-learn.org/1.1/modules/svm.html#regression

## Traning and Logging

In [1]:
from sklearn.datasets import load_diabetes
X, y = load_diabetes(return_X_y=True)
(X.shape, y.shape)

((442, 10), (442,))

In [2]:
(X[0], y[0])

(array([ 0.03807591,  0.05068012,  0.06169621,  0.02187239, -0.0442235 ,
        -0.03482076, -0.04340085, -0.00259226,  0.01990749, -0.01764613]),
 151.0)

In [3]:
(X[1], y[1])

(array([-0.00188202, -0.04464164, -0.05147406, -0.02632753, -0.00844872,
        -0.01916334,  0.07441156, -0.03949338, -0.06833155, -0.09220405]),
 75.0)

In [4]:
import getpass

import mlflow
from rikai_sklearn.mlflow import log_model
from sklearn import svm


mlflow_tracking_uri = "sqlite:///mlruns.db"
mlflow.set_tracking_uri(mlflow_tracking_uri)

# train a model
with mlflow.start_run() as run:
    ####
    # Part 1: Train the model and register it on MLflow
    ####
    
    model_svr = svm.SVR(epsilon=0.3).fit(X, y)
    model_nusvr = svm.NuSVR().fit(X, y)
    model_l_svr = svm.LinearSVR().fit(X, y)
    
    svr_name = f"{getpass.getuser()}_svr"
    nusvr_name = f"{getpass.getuser()}_nusvr"
    l_svr_name = f"{getpass.getuser()}_linear_svr"
    
    log_model(model_svr, registered_model_name=svr_name)
    log_model(model_nusvr, registered_model_name=nusvr_name)
    log_model(model_l_svr, registered_model_name=l_svr_name)


/Users/da/.cache/pants/named_caches/pex_root/venvs/97296c1a2aa5705046eb53ad742c9a9d598e9ce1/3a466fe3d02679c1e38c4c1cbe108770bca9adb4/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
2022/11/15 23:34:31 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
Registered model 'da_svr' already exists. Creating a new version of this model...
2022/11/15 23:34:31 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: da_svr, version 11
Created version '11' of model 'da_svr'.
/Users/da/.cache/pants/named_caches/pex_root/venvs/97296c1a2aa5705046eb53ad742c9a9d598e9ce1/3a466fe3d02679c1e38c4c1cbe108770bca9adb4/lib/python3.8/site-packages/rikai/spark/sql/codegen/mlflow_logger.py:183: UserWarning: value 

## Apply the model on the large scale dataset

In [5]:
from example import spark
from rikai.spark.sql.codegen.mlflow_logger import CONF_MLFLOW_TRACKING_URI
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "false")
spark.conf.set(CONF_MLFLOW_TRACKING_URI, mlflow_tracking_uri)
spark.sql(f"""
CREATE OR REPLACE MODEL svr USING 'mlflow:///{svr_name}';
"""
)

spark.sql(f"""
CREATE OR REPLACE MODEL nusvr USING 'mlflow:///{nusvr_name}';
"""
)

spark.sql(f"""
CREATE OR REPLACE MODEL l_svr USING 'mlflow:///{l_svr_name}';
"""
)

spark.sql("show models").show(10, vertical=False, truncate=False)

:: loading settings :: url = jar:file:/Users/da/.cache/pants/named_caches/pex_root/installed_wheels/fcaa57f02b90be772d50778078fc41c3660d5a6c43218e45b2c2aef2ec8e9d58/pyspark-3.2.2-py2.py3-none-any.whl/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/da/.ivy2/cache
The jars for the packages stored in: /Users/da/.ivy2/jars
ai.eto#rikai_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-73ddfb5f-6827-40cb-a91e-eadf65a6e33d;1.0
	confs: [default]
	found ai.eto#rikai_2.12;0.1.14 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found com.typesafe.scala-logging#scala-logging_2.12;3.9.4 in central
	found org.slf4j#slf4j-api;1.7.30 in spark-list
	found org.mlflow#mlflow-client;1.21.0 in central
	found org.apache.logging.log4j#log4j-core;2.17.1 in central
:: resolution report :: resolve 216ms :: artifacts dl 8ms
	:: modules in use:
	ai.eto#rikai_2.12;0.1.14 from central in [default]
	com.typesafe.scala-logging#scala-logging_2.12;3.9.4 from central in [default]
	org.apache.logging.log4j#log4j-core;2.17.1 from central in [default]
	org.mlflow#mlflow-client;1.21.0 from central in [default]
	org.slf4j#slf4j-api;1.7.30 from spark-list in [default]
	org.x

+-----+------+-----------------------+-------+
|name |flavor|uri                    |options|
+-----+------+-----------------------+-------+
|svr  |      |mlflow:///da_svr       |       |
|nusvr|      |mlflow:///da_nusvr     |       |
|l_svr|      |mlflow:///da_linear_svr|       |
+-----+------+-----------------------+-------+



In [6]:
result = spark.sql(f"""
select
  ML_PREDICT(svr, array(0.03807591,  0.05068012,  0.06169621,  0.02187239, -0.0442235 ,
        -0.03482076, -0.04340085, -0.00259226,  0.01990749, -0.01764613)) as svr,
  ML_PREDICT(nusvr, array(0.03807591,  0.05068012,  0.06169621,  0.02187239, -0.0442235 ,
        -0.03482076, -0.04340085, -0.00259226,  0.01990749, -0.01764613)) as nusvr,
  ML_PREDICT(l_svr, array(0.03807591,  0.05068012,  0.06169621,  0.02187239, -0.0442235 ,
        -0.03482076, -0.04340085, -0.00259226,  0.01990749, -0.01764613)) as l_svr
        
"""
)

result.printSchema()
result.toPandas()

root
 |-- svr: float (nullable = true)
 |-- nusvr: float (nullable = true)
 |-- l_svr: float (nullable = true)



,svr,nusvr,l_svr
0,109.949249,109.949249,109.949249


In [7]:
spark.sql(f"""
select
  ML_PREDICT(svr, array(-0.00188202, -0.04464164, -0.05147406, -0.02632753, -0.00844872,
        -0.01916334,  0.07441156, -0.03949338, -0.06833155, -0.09220405)) as svr
"""
).toPandas()

,svr
0,106.219246


In [8]:

spark.sql("""
select  ML_PREDICT(nusvr, array(-0.00188202, -0.04464164, -0.05147406, -0.02632753, -0.00844872,
        -0.01916334,  0.07441156, -0.03949338, -0.06833155, -0.09220405)) as nusvr
""").toPandas()


,nusvr
0,106.219246


In [9]:
spark.sql("""
select ML_PREDICT(l_svr, array(-0.00188202, -0.04464164, -0.05147406, -0.02632753, -0.00844872,
        -0.01916334,  0.07441156, -0.03949338, -0.06833155, -0.09220405)) as l_svr
""").toPandas()

,l_svr
0,106.219246
